In [1]:
tokenizer_checkpoint = "bert-base-uncased"
dataset_name = "imdb"

In [6]:
from transformers import AutoModelForSequenceClassification
from chop import MaseGraph
import chop.passes as passes
from pathlib import Path

mg = MaseGraph.from_checkpoint(f"{Path.cwd()}/mase/tasks/tutorial3/tutorial_3_qat")

/rds/general/user/oa321/home/miniforge3/envs/mase_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/rds/general/user/oa321/home/mase/src/chop/ir/graph/mase_graph.py:380: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializa

In [3]:
from chop.tools import get_tokenized_dataset, get_trainer

dataset, tokenizer = get_tokenized_dataset(
    dataset=dataset_name,
    checkpoint=tokenizer_checkpoint,
    return_tokenizer=True,
)

trainer = get_trainer(
    model=mg.model,
    tokenized_dataset=dataset,
    tokenizer=tokenizer,
    evaluate_metric="accuracy",
)

# Evaluate accuracy
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

INFO     Tokenizing dataset imdb with AutoTokenizer for bert-base-uncased.
/rds/general/user/oa321/home/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation accuracy: 0.8446


In [4]:
pc = {
    "weight": {
        "sparsity": 0.5,
        "method": "l1-norm",
        "scope": "local"
    },
    "activation": {
        "sparsity": 0.5,
        "method": "l1-norm",
        "scope": "local"
    }
}

mg, _ = passes.prune_transform_pass(mg, pass_args=pc)


INFO     Pruning module: bert_encoder_layer_0_attention_self_query
INFO     Pruning module: bert_encoder_layer_0_attention_self_key
INFO     Pruning module: bert_encoder_layer_0_attention_self_value
INFO     Pruning module: bert_encoder_layer_0_attention_output_dense
INFO     Pruning module: bert_encoder_layer_0_intermediate_dense
INFO     Pruning module: bert_encoder_layer_0_output_dense
INFO     Pruning module: bert_encoder_layer_1_attention_self_query
INFO     Pruning module: bert_encoder_layer_1_attention_self_key
INFO     Pruning module: bert_encoder_layer_1_attention_self_value
INFO     Pruning module: bert_encoder_layer_1_attention_output_dense
INFO     Pruning module: bert_encoder_layer_1_intermediate_dense
INFO     Pruning module: bert_encoder_layer_1_output_dense
INFO     Pruning module: bert_pooler_dense
INFO     Pruning module: classifier


In [5]:
trainer = get_trainer(
    model=mg.model,
    tokenized_dataset=dataset,
    tokenizer=tokenizer,
    evaluate_metric='accuracy',
    num_train_epochs=5
)

eval_results = trainer.evaluate()
print(f"Evaluation Accuracy: {eval_results['eval_accuracy']}")

/rds/general/user/oa321/home/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation Accuracy: 0.64804


In [6]:
trainer.train()

Step,Training Loss
500,0.459400
1000,0.419000
1500,0.424800
2000,0.397500
2500,0.405400
3000,0.412200
3500,0.419300
4000,0.398500
4500,0.389400
5000,0.388000


TrainOutput(global_step=15625, training_loss=0.39179789624023437, metrics={'train_runtime': 284.4299, 'train_samples_per_second': 439.476, 'train_steps_per_second': 54.934, 'total_flos': 0.0, 'train_loss': 0.39179789624023437, 'epoch': 5.0})

In [7]:
eval_results = trainer.evaluate()
print(f"Evaluation accuracy: {eval_results['eval_accuracy']}")

Evaluation accuracy: 0.83608
